In [1]:
from comet_ml import Experiment
#from comet_ml.integration.pytorch import log_model
import numpy as np
import os
import torch
from concrete_scars_dataset import ConcreteScarsDataset
from concrete_scars_model import ConcreteScarsModel

API_KEY = os.environ['COMETKEY']
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# experiment = Experiment(
#   api_key = API_KEY,
#   project_name = 'concrete-scars-detection',
#   workspace='my-projects'
# )

hyper_params = {
    'learning_rate': 0.001,
    'epochs': 10,
    'batch_size': 10
}
#experiment.log_parameters(hyper_params)

In [3]:
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

dataset = ConcreteScarsDataset(transform=ToTensor(),n_negative=30, n_positive=70) #default distribution 300-negative 700-positive
data_loader = DataLoader(dataset=dataset, batch_size=hyper_params['batch_size'], shuffle=True, num_workers=1)
steps = len(dataset)

model = ConcreteScarsModel().to(DEVICE)
#experiment.log_model(experiment, model, model_name='ConcreteScarsDetection')
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])

In [4]:
start = torch.cuda.Event(enable_timing=True)
#first epoch is always a few seconds longer, we dont do warmup
end = torch.cuda.Event(enable_timing=True)

for epoch in range(hyper_params['epochs']):
    error = np.zeros(steps)
    s_per_epoch = np.zeros(hyper_params['epochs'])

    start.record()
    for i, (images, masks) in enumerate(data_loader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        error[i] = loss.item()
        print(f'Step:  {(i+1)*hyper_params["batch_size"]}/{steps} epoch: {epoch+1}', end='\r', flush=True)
    end.record()
    torch.cuda.synchronize()
    
    s_per_epoch[epoch] = round(start.elapsed_time(end)/1000,3) #convert to seconds
    print(f'Epoch: {epoch+1} error: {np.mean(error)}')
    print(f'Finished in: {s_per_epoch[epoch]}s, est. time left: {round((np.average(s_per_epoch[s_per_epoch!=0])*(hyper_params["epochs"]-epoch))/60, 2)} min')
    

Epoch: 1 error: 0.002122278078459203
Finished in: 6.472s, est. time left: 1.08 min
Epoch: 2 error: 0.0008574112365022302
Finished in: 5.475s, est. time left: 0.82 min
Epoch: 3 error: 0.0008574547525495291
Finished in: 5.426s, est. time left: 0.72 min
Epoch: 4 error: 0.0008571415627375245
Finished in: 5.405s, est. time left: 0.63 min


KeyboardInterrupt: 

Showing results

In [ ]:
import matplotlib.pyplot as plt
dataiter = iter(data_loader)
image, mask = next(dataiter)

image = image.to(DEVICE)
mask = mask.to(DEVICE)
predict = model(image)

In [ ]:
with torch.no_grad():
    i = 5 # i in range(0, batch_size-1)
    img_dim = image[i].permute(1, 2, 0)
    predict_dim = predict[i].permute(1,2,0)
    msk_dim = mask[i].permute(1,2,0)
    plt.imshow(img_dim.cpu())
    plt.show()
    plt.imshow(predict_dim.cpu())
    plt.show()
    plt.imshow(msk_dim.cpu()/255)
